In [40]:
from collections import deque

import numpy as np
import matplotlib.pyplot as plt

In [41]:
CSV_DIR = "C001H001S0002/"
data_list = deque([None, None])

In [42]:
s = 20  # search window

In [43]:
class Vector2D(object):
    def __init__(self):
        self.x = 0.0
        self.y = 0.0

class TP(object):
    def __init__(self, idx):
        self.idx = idx
        self.p1 = 0
        self.p2 = 0
        self.err = 0.0
        self.flag = False

In [44]:
def nearest(imgs):
    pp0, pp1 = imgs[0], imgs[1]

    if pp0 is None or pp1 is None:
        return None

    result = np.zeros([pp0.shape[0], 4])

    dx = [Vector2D for _ in range(1)]
    pp12 = [TP(idx=x) for x in range(pp0.shape[0])]
    npa = -1  # the number of available particle
    ntsp = 0  # the number for tracking the same particle

    for ii in range(pp0.shape[0]):  # 1st
        flag = True  # init flag for the particle tracking
        c, cc = 1e10, 1e10  # arbitrary large number

        for jj in range(pp1.shape[0]):  # 2nd
            dx[0].x = pp1[jj][0] - pp0[ii][0]
            dx[0].y = pp1[jj][1] - pp0[ii][1]

            cc = np.sqrt(dx[0].x ** 2 + dx[0].y ** 2)

            if cc <= s and cc < c:
                c = cc

                if flag:
                    npa += 1

                pp12[npa].p1 = ii
                pp12[npa].p2 = jj
                pp12[npa].err = c
                pp12[npa].flag = True
                flag = False

    # post-processing
    # If different particles track the same particle,
    # giving an error flag to particle which has larger error.
    for ii in range(npa):
        if pp12[ii].flag:
            for jj in range(ii+1, npa):
                if pp12[ii].p2 == pp12[jj].p2:
                    if pp12[ii].err > pp12[jj].err:
                        pp12[ii].flag = False
                        ntsp += 1
                        break
                    else:
                        pp12[jj].flag = False
                        ntsp += 1

    print(f"The number for tracking the same particle: {ntsp}")
    print(f"The number of remaining particle by post processing: {npa - ntsp}\n")

    for ii in range(npa):
        if pp12[ii].flag:
            result[ii][0] = pp0[pp12[ii].p1][0]
            result[ii][1] = pp0[pp12[ii].p1][1]
            result[ii][2] = pp1[pp12[ii].p2][0] - pp0[pp12[ii].p1][0]
            result[ii][3] = pp1[pp12[ii].p2][1] - pp0[pp12[ii].p1][1]

    return result[:npa]


for i in range(3000, 3100, 5):
    data_list.popleft()
    tmp = np.loadtxt(CSV_DIR + f"{i:06}.csv", delimiter=',', skiprows=1)
    data_list.append(tmp)

    r = nearest(imgs=data_list)

    if r is not None:
        np.savetxt(f"nearest/n_5_{i}.csv", r, fmt='%.18f', delimiter=',', header="x, y, dx, dy")

The number for tracking the same particle: 0
The number of remaining particle by post processing: 25

The number for tracking the same particle: 1
The number of remaining particle by post processing: 26

The number for tracking the same particle: 0
The number of remaining particle by post processing: 26

The number for tracking the same particle: 0
The number of remaining particle by post processing: 24

The number for tracking the same particle: 0
The number of remaining particle by post processing: 22

The number for tracking the same particle: 0
The number of remaining particle by post processing: 21

The number for tracking the same particle: 0
The number of remaining particle by post processing: 21

The number for tracking the same particle: 0
The number of remaining particle by post processing: 22

The number for tracking the same particle: 1
The number of remaining particle by post processing: 22

The number for tracking the same particle: 1
The number of remaining particle by p